# YOLO Training in Google Colab

This notebook demonstrates how to train the YOLO model using Google Colab's GPU.

In [ ]:
# Clone repository
!git clone https://github.com/emad-siddiq/yolo-pytorch.git
%cd yolo-pytorch

# Install requirements
!pip install -r requirements.txt

# Add to Python path
import sys
sys.path.append('/content/yolo-pytorch')

In [ ]:
# Import necessary modules
from src.models.yolo import YOLOv1
from src.models.loss import YOLOLoss
from src.data.dataset import YOLODataset
from src.utils.train import train_yolo
from src.config import Config

import torch
from torch.utils.data import DataLoader

In [ ]:
# Mount Google Drive (optional, for saving weights)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Initialize model and move to GPU
model = YOLOv1(
    split_size=Config.GRID_SIZE,
    num_boxes=Config.NUM_BOXES,
    num_classes=Config.NUM_CLASSES
).to(Config.DEVICE)

# Initialize loss and optimizer
criterion = YOLOLoss()
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=Config.LEARNING_RATE,
    weight_decay=Config.WEIGHT_DECAY
)

In [ ]:
# Create datasets and dataloaders
train_dataset = YOLODataset(
    Config.TRAIN_DIR,
    S=Config.GRID_SIZE,
    B=Config.NUM_BOXES,
    C=Config.NUM_CLASSES
)

train_loader = DataLoader(
    train_dataset,
    batch_size=Config.BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

In [ ]:
# Train the model
train_yolo(
    model,
    train_loader,
    criterion,
    optimizer,
    Config.EPOCHS,
    Config.DEVICE
)

In [ ]:
# Save model weights
torch.save(model.state_dict(), 'data/weights/yolo_model.pth')

# Save to Google Drive (optional)
# !cp data/weights/yolo_model.pth '/content/drive/MyDrive/yolo_model.pth'

In [ ]:
# Commit and push changes to GitHub
!git add .
!git commit -m "Update model weights and training results"
!git push origin main